In [25]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd
import shutil
import torch


c:\Users\Jonathan van Zyl\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Neural Network Model

In [2]:
def apply_camera_model(predicted_params, points_3d):
    """
    Apply the camera model to project 3D points to 2D using predicted camera parameters.
    
    :param predicted_params: Tensor containing the predicted camera parameters [f_x, f_y, c_x, c_y, k1, k2, k3, p_1, p_2]
    :param points_3d: Tensor containing 3D points in camera coordinates, shape [N, 3]
    :return: Tensor containing the projected 2D points, shape [N, 2]
    """
    # Unpack predicted parameters
    predicted_params = predicted_params.squeeze()
    # Unpack predicted parameters
    f_x, f_y, c_x, c_y, k1, k2, k3, p_1, p_2 = predicted_params

    f_x =  f_x
    f_y = f_y 
    c_x = c_x 
    c_y = c_y 

    # Unpack K matrix
    
    # Normalize 3D points (X_c/Z_c, Y_c/Z_c)
    X_c = points_3d[:, 0]
    Y_c = points_3d[:, 1]
    Z_c = points_3d[:, 2]
    x_normalized = X_c / Z_c
    y_normalized = Y_c / Z_c
    
    # Compute r^2 = x_normalized^2 + y_normalized^2
    r_squared = x_normalized**2 + y_normalized**2
    
    # Apply radial distortion correction
    radial_factor = 1 + k1 * r_squared + k2 * r_squared**2 + k3 * r_squared**3
    x_radial = x_normalized * radial_factor
    y_radial = y_normalized * radial_factor
    
    # Apply tangential distortion correction
    x_tangential = 2 * p_1 * x_normalized * y_normalized + p_2 * (r_squared + 2 * x_normalized**2)
    y_tangential = p_1 * (r_squared + 2 * y_normalized**2) + 2 * p_2 * x_normalized * y_normalized
    
    # Combine radial and tangential distortions
    x_distorted = x_radial + x_tangential
    y_distorted = y_radial + y_tangential
    
    # Convert distorted coordinates back to pixel coordinates
    u = f_x * x_distorted + c_x
    v = f_y * y_distorted + c_y
    
    # Stack the 2D points into a single tensor
    points_2d = torch.stack([u, v], dim=-1)

    img_width = 240
    img_height = 520

    # Normalise the points
    #points_2d[:, 0] = points_2d[:, 0] / img_width
    #points_2d[:, 1] = points_2d[:, 1] / img_height
    
    return points_2d

In [22]:
# Load x y z from the csv file
points3D_drone_XYZ = np.genfromtxt('Translated.csv', delimiter=',', skip_header=1, usecols=(1, 2, 3)) # (x, y, z) : (N, 3)
# Read the CSV file
df = pd.read_csv('Translated.csv')

# Extract the image names
image_name_list = df['image_name'].values




In [29]:
# Extract K and D
K = camera_front.K
D = camera_front.D

# Extract f_x, f_y, c_x, c_y, k1, k2, k3, p_1, p_2
f_x = K[0, 0]
f_y = K[1, 1]
c_x = K[0, 2]
c_y = K[1, 2]
k1 = D[0]
k2 = D[1]
k3 = D[2]
p_1 = D[3]
p_2 = 0

# Create a tensor containing the predicted camera parameters
predicted_params = torch.tensor([f_x, f_y, c_x, c_y, k1, k2, k3, p_1, p_2])

# Create a tensor containing the 3D points
points_3d = torch.tensor(points3D_cyberzoo_camera_XYZRBG[:, :3])

# Apply the camera model
points_2d = apply_camera_model(predicted_params, points_3d)

# Convert to numpy
points_2d_NN = points_2d.numpy()

print(points_2d_NN)

[[ 1.64703428e+03  4.79957524e+03]
 [-6.60659916e+02  1.18945498e+03]
 [-1.92157999e+03  9.50606024e+02]
 [-1.07777556e+04  2.22764128e+03]
 [-1.24485478e+03  4.16365330e+01]
 [-3.41428405e+02 -8.92841553e+01]
 [-7.01128465e+02 -6.12227531e+02]
 [ 1.88995976e+00 -2.76816471e+01]
 [ 4.10137386e+01 -1.42047278e+01]
 [ 1.20137212e+03 -7.38969685e+01]
 [ 2.58090625e+02  5.85127207e+01]
 [ 4.85184174e+02  4.03285228e+02]
 [ 1.64703428e+03  4.79957524e+03]]


<ipython-input-29-02e047603fa1>:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_params = torch.tensor([f_x, f_y, c_x, c_y, k1, k2, k3, p_1, p_2])


OOP Model

In [20]:
make_image_folder = False
if make_image_folder:
    # Load the CSV file into a DataFrame

    # Specify the directory where the images are located
    image_dir = r"C:\Users\Jonathan van Zyl\Documents\GitHub\paparazzi\Data_gitignore\AE4317_2019_datasets\cyberzoo_poles_panels\20190121-140205"
    # Create a list to store the paths of the images
    image_paths = []

    # Iterate over the image names in the DataFrame
    for image_name in image_name_list:
        # Construct the full path of the image
        image_path_full = os.path.join(image_dir, image_name)
        image_path = image_name
        
        # Check if the image exists
        if os.path.exists(image_path_full):
            # If the image exists, add its path to the list
            image_paths.append(image_path)
        else:
            print(f"Image {image_name} not found.")



    # Add these images to a folder called Images
    folder_path =r"C:\Users\Jonathan van Zyl\Documents\GitHub\paparazzi\COMPUTERVISIONSIM\Projection\Images"

    # Add images to folder

    for image_path in image_paths:
        # Construct the full path of the image
        image_path_full = os.path.join(image_dir, image_path)
        image_path = image_name
        
        # Check if the image exists
        if os.path.exists(image_path_full):
            # If the image exists, add its path to the list
            image_paths.append(image_path)
        else:
            print(f"Image {image_name} not found.")
            
        # Copy the image to the folder
        shutil.copy(image_path_full, folder_path)


['100049163.jpg' '100282495.jpg' '100482507.jpg' '100715833.jpg'
 '100982494.jpg' '101215830.jpg' '101449184.jpg' '101682492.jpg'
 '101915820.jpg' '102149155.jpg' '102382486.jpg' '102615813.jpg'
 '102849149.jpg' '103082474.jpg' '103315832.jpg' '103549134.jpg'
 '103782482.jpg' '103982472.jpg' '104215805.jpg' '104449137.jpg'
 '104682471.jpg' '104915799.jpg' '105149136.jpg' '105382468.jpg'
 '105615793.jpg' '105849117.jpg' '106049124.jpg' '106282449.jpg'
 '106515776.jpg' '106715785.jpg' '106949116.jpg' '107182448.jpg'
 '107415764.jpg' '107649095.jpg' '107882434.jpg' '108115758.jpg'
 '108349084.jpg' '108582419.jpg' '108815749.jpg' '109049081.jpg'
 '109282416.jpg' '109515746.jpg' '109749075.jpg' '109949080.jpg'
 '110182409.jpg' '110449070.jpg' '110649074.jpg' '110882428.jpg'
 '111115735.jpg' '111349063.jpg' '111549065.jpg' '111782398.jpg'
 '112015728.jpg' '112282388.jpg' '112515718.jpg' '112749047.jpg'
 '112949055.jpg' '113182384.jpg' '113449057.jpg' '60582822.jpg'
 '60849474.jpg' '61082806.

KeyboardInterrupt: 

In [37]:
# Use the Camera class from projection_functions.py
from projection_functions import Camera, StateVector, CyberZooStructure, VideoFeed

image_path = r"C:\Users\Jonathan van Zyl\Documents\GitHub\paparazzi\Data_gitignore\AE4317_2019_datasets\cyberzoo_poles_panels\20190121-140205"
image_path = r"C:\Users\Jonathan van Zyl\Documents\GitHub\paparazzi\COMPUTERVISIONSIM\Projection\Images"
file_path = r"C:\Users\Jonathan van Zyl\Documents\GitHub\paparazzi\Data_gitignore\AE4317_2019_datasets\cyberzoo_poles_panels\20190121-140303.csv"


# Create a camera object
camera_front = Camera()

# Create a state vector object
state_vector = StateVector(file_path)

zmin = max(state_vector.z_pos_array)#max(state_vector.z_pos_array)
cyberzoo = CyberZooStructure(zmin)
points3d_cyberzoo = cyberzoo.return_points3d()
formatted_colored_points = cyberzoo.get_colored_perimeter_points()

# Load the image
images = VideoFeed(image_path)
# Rotate the image
images.image_rotate_90_counter()

# Loop over the images
x_pos_camera = []
y_pos_camera = []
z_pos_camera = []
theta_camera = []
phi_camera = []
psi_camera = []
times_list = []

for i in range(len(images.frame_files)):
    images.index = i
    images.image_current = images.image_read(i)
    images.image_rotate_90_counter()

    time = images.find_time()
    times_list.append(time)
    # update the camera object
    camera_front.update_state_vector(state_vector, time)
    x_pos_camera.append(camera_front.x_pos); y_pos_camera.append(camera_front.y_pos); z_pos_camera.append(camera_front.z_pos)
    theta_camera.append(camera_front.theta); phi_camera.append(camera_front.phi); psi_camera.append(camera_front.psi)


    # Create the projection
    # Project the cyberzoo points
    # Convert points3d_cyberzoo elements to a numpy array of type float32 and reshape to have 3 channels
    points2D_cyberzoo_XYRGB, points3D_cyberzoo_camera_XYZRBG = camera_front.project_3D_to_2D(np.array(formatted_colored_points, dtype=np.float32).reshape(-1, 1, 6), fisheye_bool = True)
    images.draw_circle(points2D_cyberzoo_XYRGB,radius=10)
    
    # Draw the points black
    images.draw_circle(points_2d_current_xyzrgb)

    # Show the image
    images.image_show(waitKeyvalue = 1)
        


[[395288, 2495779, 0, 0, 0]]
[[545.4453125  265.39138794   0.         255.           0.        ]
 [-59.2143364  137.91688538  85.         170.           0.        ]
 [ 11.61762714 135.82585144 170.          85.           0.        ]
 [131.31980896 153.54528809 255.           0.           0.        ]
 [242.15080261 133.84861755 255.           0.           0.        ]
 [424.72970581 117.41544342 127.5          0.         127.5       ]
 [584.48126221 125.00906372   0.           0.         255.        ]
 [189.69245911 242.00878906   0.           0.         255.        ]
 [ 64.96611786 341.12765503 127.5        127.5        127.5       ]
 [223.09028625 292.6171875  255.         255.           0.        ]
 [406.85073853 311.96310425 255.         255.           0.        ]
 [579.69104004 305.74359131 127.5        255.           0.        ]
 [545.4453125  265.39138794   0.         255.           0.        ]]


IndexError: invalid index to scalar variable.

: 

Run NN model